In [3]:
import ccxt
import dontshare_config as ds
import pandas as pd
import backtrader as bt
import matplotlib.pyplot as pt
import datetime

In [4]:
#Authentification
exchange = ccxt.mexc({
    'enableRateLimit': True,
    'apiKey': ds.API_KEY, 
    'secret': ds.SECRET_KEY
    })

In [7]:
# Define the trading pair and time frame
symbol = 'BTC/USDT'
timeframe = '1h'  # 1-hour candles

# Fetch historical OHLCV (Open, High, Low, Close, Volume) data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe)

# Convert the data into a DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Calculate moving averages using pandas
df['EMA1'] = df['close'].rolling(window=10).mean()  # 10-period moving average
df['EMA2'] = df['close'].rolling(window=50).mean()  # 30-period moving average

# Print or use the moving average values
print(df[['close', 'EMA1', 'EMA2']])

# Your signal generator function for moving average crossovers
def signal_generator_ma(df):
    short_ma = df['EMA1']
    long_ma = df['EMA2']

    # Golden Cross: Short-term MA crosses above Long-term MA
    if short_ma[-1] > long_ma[-1]: #and short_ma.shift(1)[-1] <= long_ma.shift(1)[-1]:
        return 1

    # Death Cross: Short-term MA crosses below Long-term MA
    elif short_ma[-1] < long_ma[-1]: #and short_ma.shift(1)[-1] >= long_ma.shift(1)[-1]:
        return -1

    # No clear crossover
    else:
        return 0

# Apply the signal generator function to the DataFrame
signal_ma = [0]
for i in range(1, len(df)):
    df_slice = df.iloc[i - 1:i + 1]
    signal_ma.append(signal_generator_ma(df_slice))

# Add the signal column to the DataFrame
df["signal_ma"] = signal_ma

# Display the DataFrame
print(df[['close', 'signal_ma']])



                        close       EMA1        EMA2
timestamp                                           
2023-10-30 03:00:00  34270.42        NaN         NaN
2023-10-30 04:00:00  34322.22        NaN         NaN
2023-10-30 05:00:00  34250.99        NaN         NaN
2023-10-30 06:00:00  34301.18        NaN         NaN
2023-10-30 07:00:00  34195.68        NaN         NaN
...                       ...        ...         ...
2023-11-19 18:00:00  36836.00  36561.496  36507.5016
2023-11-19 19:00:00  36948.96  36593.605  36516.4808
2023-11-19 20:00:00  36916.81  36632.234  36525.3730
2023-11-19 21:00:00  36984.46  36685.318  36538.6144
2023-11-19 22:00:00  36981.68  36734.223  36548.6228

[500 rows x 3 columns]
                        close  signal_ma
timestamp                               
2023-10-30 03:00:00  34270.42          0
2023-10-30 04:00:00  34322.22          0
2023-10-30 05:00:00  34250.99          0
2023-10-30 06:00:00  34301.18          0
2023-10-30 07:00:00  34195.68          0


In [14]:
# Define the trading pair and time frame
symbol = 'BTC/USDT'
timeframe = '1h'

#Define the desired date range
start_date = datetime.datetime(2023, 1, 1)  # Adjust the start date as needed
end_date = datetime.datetime(2023, 2, 10)  # Adjust the end date as needed

# Convert start_date and end_date to timestamps in seconds
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Fetch historical OHLCV (Open, High, Low, Close, Volume) data
ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start_timestamp * 1000, limit=None)

# Convert the data into a Pandas DataFrame
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Filter data based on the desired date range
df = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)]

# Convert the data into a Pandas DataFrame
data = bt.feeds.PandasData(dataname=df.set_index('timestamp'), openinterest=None)

# Define the Backtrader strategy
class MyStrategy(bt.Strategy):
    params = (
        ('short_period', 5),
        ('long_period', 10),
    )

    def __init__(self):
        self.short_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.short_period)
        self.long_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.long_period)

    def next(self):
        if self.short_ma > self.long_ma and self.position.size == 0:
            self.buy()

        elif self.short_ma < self.long_ma and self.position.size == 0:
            self.sell()

# Create a Backtrader Cerebro engine
cerebro = bt.Cerebro()

# Add the data feed to the engine
cerebro.adddata(data)

# Add the strategy to the engine
cerebro.addstrategy(MyStrategy)

# Set the initial cash amount
cerebro.broker.set_cash(10000)

# Set the commission
cerebro.broker.setcommission(commission=0.001)

# Add a portfolio value observer
cerebro.addobserver(bt.observers.Value)

# Add a plot of the portfolio value
#cerebro.addobserver(bt.observers.BuySell)
#cerebro.addobserver(bt.observers.Broker)
#cerebro.addobserver(bt.observers.Trades)
#cerebro.addobserver(bt.observers.DrawDown)

# Print the starting cash amount
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
cerebro.run()

# Print the final cash amount
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

print('Start Date:', df['timestamp'].iloc[0])
print('End Date:', df['timestamp'].iloc[-1])


Starting Portfolio Value: 10000.00
Ending Portfolio Value: 3272.53
Start Date: 2023-01-01 00:00:00
End Date: 2023-01-21 18:00:00
